### Import libraries and necessary datasets

In [383]:
import psycopg2
import pandas as pd
import numpy as np

In [384]:
df_wg = pd.read_csv('full_database.csv')
df_wg.shape

(39202, 5)

In [385]:
df_wg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39202 entries, 0 to 39201
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   education_attained  39202 non-null  int64  
 1   enrollment_status   39202 non-null  int64  
 2   age                 39202 non-null  float64
 3   sample_weight       39202 non-null  float64
 4   employed            39202 non-null  int64  
dtypes: float64(2), int64(3)
memory usage: 1.5 MB


In [386]:
youth = df_wg[(df_wg.age >=16) & (df_wg.age <=24)]
youth.head()

,education_attained,enrollment_status,age,sample_weight,employed
13,21,0,24.0,20.0,1
14,21,0,24.0,15.0,1
28,18,0,19.0,30.0,0
31,13,0,17.0,36.0,0
37,16,1,19.0,36.0,0


In [387]:
#add ranges for binning of age groups
youth['age_range'] = (youth.age -16)//3
youth.head()

<ipython-input-387-3603be6b57db>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youth['age_range'] = (youth.age -16)//3


,education_attained,enrollment_status,age,sample_weight,employed,age_range
13,21,0,24.0,20.0,1,2.0
14,21,0,24.0,15.0,1,2.0
28,18,0,19.0,30.0,0,1.0
31,13,0,17.0,36.0,0,0.0
37,16,1,19.0,36.0,0,1.0


## Total Opportunity Youth
Pulling the total number of opportunity youth as defined by: "not in school, not working, between the ages of 16 and 24" from 2016 report.

In [388]:
youth

,education_attained,enrollment_status,age,sample_weight,employed,age_range
13,21,0,24.0,20.0,1,2.0
14,21,0,24.0,15.0,1,2.0
28,18,0,19.0,30.0,0,1.0
31,13,0,17.0,36.0,0,0.0
37,16,1,19.0,36.0,0,1.0
...,...,...,...,...,...,...
39162,11,0,18.0,20.0,0,0.0
39166,19,0,20.0,17.0,0,1.0
39188,19,0,19.0,6.0,0,1.0
39190,18,0,18.0,14.0,0,0.0


In [389]:
no_diploma = (youth['education_attained'] < 16)
not_enrolled = (youth['enrollment_status'] == 0)
not_employed = (youth['employed'] == 0)

youth['opportunity_youth'] = 0
youth.loc[not_enrolled & not_employed, 'opportunity_youth'] = 1

youth.head()
#youth['opportunity_youth'] = 0
#youth.loc[(youth['enrollment_status'] == 0) & (youth['employed'] == 0), 'opportunity_youth'] = 1

<ipython-input-389-d24c649821a9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youth['opportunity_youth'] = 0
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,education_attained,enrollment_status,age,sample_weight,employed,age_range,opportunity_youth
13,21,0,24.0,20.0,1,2.0,0
14,21,0,24.0,15.0,1,2.0,0
28,18,0,19.0,30.0,0,1.0,1
31,13,0,17.0,36.0,0,0.0,1
37,16,1,19.0,36.0,0,1.0,0


In [390]:
youth[youth['opportunity_youth'] == 1].count()

education_attained    1798
enrollment_status     1798
age                   1798
sample_weight         1798
employed              1798
age_range             1798
opportunity_youth     1798
dtype: int64

In [391]:
youth

,education_attained,enrollment_status,age,sample_weight,employed,age_range,opportunity_youth
13,21,0,24.0,20.0,1,2.0,0
14,21,0,24.0,15.0,1,2.0,0
28,18,0,19.0,30.0,0,1.0,1
31,13,0,17.0,36.0,0,0.0,1
37,16,1,19.0,36.0,0,1.0,0
...,...,...,...,...,...,...,...
39162,11,0,18.0,20.0,0,0.0,1
39166,19,0,20.0,17.0,0,1.0,1
39188,19,0,19.0,6.0,0,1.0,1
39190,18,0,18.0,14.0,0,0.0,1


## Total Working Without a Diploma (and also out of school) 



In [392]:
youth['working_without_diploma'] = 0
youth.loc[no_diploma & (youth['employed'] == 1) & not_enrolled,'working_without_diploma'] = 1
youth

<ipython-input-392-95c571cfafff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youth['working_without_diploma'] = 0
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,education_attained,enrollment_status,age,sample_weight,employed,age_range,opportunity_youth,working_without_diploma
13,21,0,24.0,20.0,1,2.0,0,0
14,21,0,24.0,15.0,1,2.0,0,0
28,18,0,19.0,30.0,0,1.0,1,0
31,13,0,17.0,36.0,0,0.0,1,0
37,16,1,19.0,36.0,0,1.0,0,0
...,...,...,...,...,...,...,...,...
39162,11,0,18.0,20.0,0,0.0,1,0
39166,19,0,20.0,17.0,0,1.0,1,0
39188,19,0,19.0,6.0,0,1.0,1,0
39190,18,0,18.0,14.0,0,0.0,1,0


## Opportunity Youth No Diploma 
I ALSO TRIED TO DO IT FROM LARGER DATA SET INSTEAD OF FROM SUBSET, results were the same

In [393]:
youth['diploma'] = 1
youth.loc[no_diploma,'diploma'] = 0
youth

<ipython-input-393-1fabab18d87f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youth['diploma'] = 1
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,education_attained,enrollment_status,age,sample_weight,employed,age_range,opportunity_youth,working_without_diploma,diploma
13,21,0,24.0,20.0,1,2.0,0,0,1
14,21,0,24.0,15.0,1,2.0,0,0,1
28,18,0,19.0,30.0,0,1.0,1,0,1
31,13,0,17.0,36.0,0,0.0,1,0,0
37,16,1,19.0,36.0,0,1.0,0,0,1
...,...,...,...,...,...,...,...,...,...
39162,11,0,18.0,20.0,0,0.0,1,0,0
39166,19,0,20.0,17.0,0,1.0,1,0,1
39188,19,0,19.0,6.0,0,1.0,1,0,1
39190,18,0,18.0,14.0,0,0.0,1,0,1


## Opportunity Youth High School Diploma or GED 

In [394]:
youth['diploma_GED'] = 0
youth.loc[youth['education_attained'].isin([16,17]), 'diploma_GED'] = 1
youth

<ipython-input-394-1cdd0bcaa8de>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youth['diploma_GED'] = 0
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,education_attained,enrollment_status,age,sample_weight,employed,age_range,opportunity_youth,working_without_diploma,diploma,diploma_GED
13,21,0,24.0,20.0,1,2.0,0,0,1,0
14,21,0,24.0,15.0,1,2.0,0,0,1,0
28,18,0,19.0,30.0,0,1.0,1,0,1,0
31,13,0,17.0,36.0,0,0.0,1,0,0,0
37,16,1,19.0,36.0,0,1.0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...
39162,11,0,18.0,20.0,0,0.0,1,0,0,0
39166,19,0,20.0,17.0,0,1.0,1,0,1,0
39188,19,0,19.0,6.0,0,1.0,1,0,1,0
39190,18,0,18.0,14.0,0,0.0,1,0,1,0


## Opportunity Youth Some College, No Degree 

In [395]:
youth['some_college_no_degree'] = 0
youth.loc[youth['education_attained'].isin([18,19]), 'some_college_no_degree'] = 1
youth

<ipython-input-395-f63010292aca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youth['some_college_no_degree'] = 0
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,education_attained,enrollment_status,age,sample_weight,employed,age_range,opportunity_youth,working_without_diploma,diploma,diploma_GED,some_college_no_degree
13,21,0,24.0,20.0,1,2.0,0,0,1,0,0
14,21,0,24.0,15.0,1,2.0,0,0,1,0,0
28,18,0,19.0,30.0,0,1.0,1,0,1,0,1
31,13,0,17.0,36.0,0,0.0,1,0,0,0,0
37,16,1,19.0,36.0,0,1.0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
39162,11,0,18.0,20.0,0,0.0,1,0,0,0,0
39166,19,0,20.0,17.0,0,1.0,1,0,1,0,1
39188,19,0,19.0,6.0,0,1.0,1,0,1,0,1
39190,18,0,18.0,14.0,0,0.0,1,0,1,0,1


## Opportunity Youth Degree(Associate or Higher)

In [396]:
youth['assoc_or_higher'] = 0
youth.loc[youth['education_attained'] >= 20, 'assoc_or_higher'] = 1
youth

<ipython-input-396-76895dc147ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youth['assoc_or_higher'] = 0
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,education_attained,enrollment_status,age,sample_weight,employed,age_range,opportunity_youth,working_without_diploma,diploma,diploma_GED,some_college_no_degree,assoc_or_higher
13,21,0,24.0,20.0,1,2.0,0,0,1,0,0,1
14,21,0,24.0,15.0,1,2.0,0,0,1,0,0,1
28,18,0,19.0,30.0,0,1.0,1,0,1,0,1,0
31,13,0,17.0,36.0,0,0.0,1,0,0,0,0,0
37,16,1,19.0,36.0,0,1.0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
39162,11,0,18.0,20.0,0,0.0,1,0,0,0,0,0
39166,19,0,20.0,17.0,0,1.0,1,0,1,0,1,0
39188,19,0,19.0,6.0,0,1.0,1,0,1,0,1,0
39190,18,0,18.0,14.0,0,0.0,1,0,1,0,1,0


# RENAME SAMPLE WEIGHT COLUMNS

# Merged Table-- Totals of OY no degree, High School/GED, some college no degree, college degree
ADD TOTAL OY COLUMN

In [397]:
total_population = pd.DataFrame()
total_population['Total Population'] = youth.groupby('age_range').sum()['sample_weight']
total_population['Opportunity Youth'] = youth[youth['opportunity_youth'] == 1].groupby('age_range').sum()['sample_weight']
total_population['Working Without Diploma'] = youth[youth['working_without_diploma'] == 1].groupby('age_range').sum()['sample_weight']
total_population['Not an Opportunity Youth'] = youth[youth['opportunity_youth'] == 0].groupby('age_range').sum()['sample_weight']

total_population

,Total Population,Opportunity Youth,Working Without Diploma,Not an Opportunity Youth
age_range,,,,
0.0,33791.0,23782.0,4571.0,10009.0
1.0,29915.0,11443.0,1641.0,18472.0
2.0,36277.0,7644.0,1526.0,28633.0


In [398]:
oy = youth[youth['opportunity_youth'] == 1]

In [399]:
opportunity_youth = pd.DataFrame()
opportunity_youth['Opportunity Youth'] =oy.groupby('age_range').sum()['sample_weight']
opportunity_youth['No diploma'] = oy[oy.diploma == 0].groupby('age_range').sum()['sample_weight']
opportunity_youth['HS diploma or GED'] = oy[oy['diploma_GED'] == 1].groupby('age_range').sum()['sample_weight']
opportunity_youth['Some College, no degree'] = oy[oy['some_college_no_degree'] == 1].groupby('age_range').sum()['sample_weight']
opportunity_youth['Degree (Associate or higher)'] = oy[oy['assoc_or_higher'] == 1].groupby('age_range').sum()['sample_weight']
opportunity_youth.fillna(0, inplace = True)
opportunity_youth

,Opportunity Youth,No diploma,HS diploma or GED,"Some College, no degree",Degree (Associate or higher)
age_range,,,,,
0.0,23782.0,20487.0,2214.0,1022.0,59.0
1.0,11443.0,2199.0,3570.0,4654.0,1020.0
2.0,7644.0,1440.0,2282.0,2310.0,1612.0


In [400]:
total_population

,Total Population,Opportunity Youth,Working Without Diploma,Not an Opportunity Youth
age_range,,,,
0.0,33791.0,23782.0,4571.0,10009.0
1.0,29915.0,11443.0,1641.0,18472.0
2.0,36277.0,7644.0,1526.0,28633.0


In [401]:
for col in opportunity_youth.columns:
    opportunity_youth[col + '%'] = (100*opportunity_youth[col]/opportunity_youth['Opportunity Youth']).round(2)
    opportunity_youth[col + '%'] = opportunity_youth[col + '%'].astype(str) + '%'
opportunity_youth

,Opportunity Youth,No diploma,HS diploma or GED,"Some College, no degree",Degree (Associate or higher),Opportunity Youth%,No diploma%,HS diploma or GED%,"Some College, no degree%",Degree (Associate or higher)%
age_range,,,,,,,,,,
0.0,23782.0,20487.0,2214.0,1022.0,59.0,100.0%,86.14%,9.31%,4.3%,0.25%
1.0,11443.0,2199.0,3570.0,4654.0,1020.0,100.0%,19.22%,31.2%,40.67%,8.91%
2.0,7644.0,1440.0,2282.0,2310.0,1612.0,100.0%,18.84%,29.85%,30.22%,21.09%


In [402]:
col1 = opportunity_youth.columns
col1

Index(['Opportunity Youth', 'No diploma', 'HS diploma or GED',
       'Some College, no degree', 'Degree (Associate or higher)',
       'Opportunity Youth%', 'No diploma%', 'HS diploma or GED%',
       'Some College, no degree%', 'Degree (Associate or higher)%'],
      dtype='object')

In [403]:
col1 = ['Opportunity Youth','Opportunity Youth%',
        'No diploma','No diploma%', 
        'HS diploma or GED', 'HS diploma or GED%',
       'Some College, no degree', 'Some College, no degree%',
       'Degree (Associate or higher)','Degree (Associate or higher)%']
opportunity_youth = opportunity_youth[col1]
opportunity_youth

,Opportunity Youth,Opportunity Youth%,No diploma,No diploma%,HS diploma or GED,HS diploma or GED%,"Some College, no degree","Some College, no degree%",Degree (Associate or higher),Degree (Associate or higher)%
age_range,,,,,,,,,,
0.0,23782.0,100.0%,20487.0,86.14%,2214.0,9.31%,1022.0,4.3%,59.0,0.25%
1.0,11443.0,100.0%,2199.0,19.22%,3570.0,31.2%,4654.0,40.67%,1020.0,8.91%
2.0,7644.0,100.0%,1440.0,18.84%,2282.0,29.85%,2310.0,30.22%,1612.0,21.09%


In [404]:
for col in total_population.columns:
    total_population[col + '%'] = (100*total_population[col]/total_population['Total Population']).round(2)
    total_population[col + '%'] = total_population[col + '%'].astype(str) + '%'
total_population

,Total Population,Opportunity Youth,Working Without Diploma,Not an Opportunity Youth,Total Population%,Opportunity Youth%,Working Without Diploma%,Not an Opportunity Youth%
age_range,,,,,,,,
0.0,33791.0,23782.0,4571.0,10009.0,100.0%,70.38%,13.53%,29.62%
1.0,29915.0,11443.0,1641.0,18472.0,100.0%,38.25%,5.49%,61.75%
2.0,36277.0,7644.0,1526.0,28633.0,100.0%,21.07%,4.21%,78.93%


In [405]:
col2 = total_population.columns
col2

Index(['Total Population', 'Opportunity Youth', 'Working Without Diploma',
       'Not an Opportunity Youth', 'Total Population%', 'Opportunity Youth%',
       'Working Without Diploma%', 'Not an Opportunity Youth%'],
      dtype='object')

In [406]:
col2 = ['Total Population', 'Total Population%',
        'Opportunity Youth','Opportunity Youth%', 
        'Working Without Diploma', 'Working Without Diploma%',
       'Not an Opportunity Youth','Not an Opportunity Youth%']
total_population = total_population[col2]
total_population

,Total Population,Total Population%,Opportunity Youth,Opportunity Youth%,Working Without Diploma,Working Without Diploma%,Not an Opportunity Youth,Not an Opportunity Youth%
age_range,,,,,,,,
0.0,33791.0,100.0%,23782.0,70.38%,4571.0,13.53%,10009.0,29.62%
1.0,29915.0,100.0%,11443.0,38.25%,1641.0,5.49%,18472.0,61.75%
2.0,36277.0,100.0%,7644.0,21.07%,1526.0,4.21%,28633.0,78.93%


In [407]:
total_population.reset_index(inplace = True)

In [408]:
ages = ['16-18','19-21','22-24']

In [409]:
total_population.reset_index(inplace = True)
total_population.loc[:,'ages'] = ages
total_population.set_index('ages',inplace = True)
total_population.drop(columns = ['age_range', 'index'], inplace = True)

total_population

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Total Population,Total Population%,Opportunity Youth,Opportunity Youth%,Working Without Diploma,Working Without Diploma%,Not an Opportunity Youth,Not an Opportunity Youth%
ages,,,,,,,,
16-18,33791.0,100.0%,23782.0,70.38%,4571.0,13.53%,10009.0,29.62%
19-21,29915.0,100.0%,11443.0,38.25%,1641.0,5.49%,18472.0,61.75%
22-24,36277.0,100.0%,7644.0,21.07%,1526.0,4.21%,28633.0,78.93%


In [410]:
opportunity_youth.reset_index(inplace = True)
opportunity_youth.loc[:,'ages'] = ages
opportunity_youth.set_index('ages',inplace = True)
opportunity_youth.drop(columns = ['age_range'], inplace = True)
opportunity_youth

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Opportunity Youth,Opportunity Youth%,No diploma,No diploma%,HS diploma or GED,HS diploma or GED%,"Some College, no degree","Some College, no degree%",Degree (Associate or higher),Degree (Associate or higher)%
ages,,,,,,,,,,
16-18,23782.0,100.0%,20487.0,86.14%,2214.0,9.31%,1022.0,4.3%,59.0,0.25%
19-21,11443.0,100.0%,2199.0,19.22%,3570.0,31.2%,4654.0,40.67%,1020.0,8.91%
22-24,7644.0,100.0%,1440.0,18.84%,2282.0,29.85%,2310.0,30.22%,1612.0,21.09%


In [411]:
opportunity_youth.to_csv('opportunity_youth.csv', index = False)
total_population.to_csv('total_population.csv', index = False)